In [1]:
# cd /stanford-corenlp-full-2018-10-05
# java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer
from pycorenlp import StanfordCoreNLP
import csv
import os
import pandas as pd
import nltk
from nltk.tree import *
from nltk import Tree
from nltk.tag.stanford import StanfordPOSTagger,StanfordNERTagger
from nltk.parse.corenlp import CoreNLPParser
from nltk import word_tokenize, pos_tag
from collections import Counter
import codecs
import spacy
%load_ext autoreload
%autoreload 2

In [2]:

#jar and model parameters need to be modified with your own path
# jar = '/Users/chaowei/Downloads/stanford-postagger-full-2020-11-17/stanford-postagger.jar'
# model = '/Users/chaowei/Downloads/stanford-postagger-full-2020-11-17/models/english-left3words-distsim.tagger'
# stpos = StanfordPOSTagger(jar, model)

# stanford_classifier = '/Users/chaowei/Downloads/stanford-ner-2020-11-17/classifiers/english.all.3class.distsim.crf.ser.gz'
# stanford_ner_path = '/Users/chaowei/Downloads/stanford-ner-2020-11-17/stanford-ner-4.2.0-javadoc.jar'
# stner = StanfordNERTagger(stanford_classifier, stanford_ner_path)

First_Pronouns = ['i', 'we', 'me','us','our','ours', 'ourself', 'ourselves', 'mine']
Second_Pronouns = ['you', 'your','yours', 'yourself', 'yourselves']
Third_Pronouns = ['he','his','him','she','her','hers','it','its','they','their','theirs','them', 'themself', 'themselves','itself']
nouns_tags = ['NN', 'NNS', 'NNP', 'NNPS']

In [3]:
def stpos_tag_check(stpos_tags, spacy_ner_tags, triple, triple_tag):
    indexes = triple[triple_tag]
    collected_person = []
    for index in range(indexes[0], indexes[1]):
        stpos_tag = stpos_tags[index]
        spacy_ner_tag = spacy_ner_tags[index] 
        if len(str(spacy_ner_tag)) != 0:  
            collected_person.append(4)              
        else:
                collected_person.append(0)
    return collected_person

In [4]:
def run_object(varible):
    if len(varible) == 0:
        return 0
    else:
        if 1 in varible:
            return 1
        elif 2 in varible:
            return 2
        elif 3 in varible:
            return 3
        elif len(Counter(varible)) > 1:
            while 0 in varible:       
                varible.remove(0)  
            #print varible,'varible'
            return Counter(varible).most_common(1)[0][0]
        else:
            return Counter(varible).most_common(1)[0][0]

In [5]:
def persons_check(triple, stpos_tags, spacy_ner_tags):
    tokenized_triple = [word_tokenize(triple['subject']),
                        word_tokenize(triple['relation']), 
                        word_tokenize(triple['object'])]
    triple_tag = ['subjectSpan', 
                 'relationSpan',
                 'objectSpan']
    subject_person = [] 
    for element in tokenized_triple[0]:       
        if element in First_Pronouns:
            subject_person.append(1)
        elif element in Second_Pronouns:
            subject_person.append(2)
        elif element in Third_Pronouns:
            subject_person.append(3)
        elif len(stpos_tag_check(stpos_tags, spacy_ner_tags, triple, triple_tag[0])) != 0:
            subject_person.append(Counter([a for a in stpos_tag_check(stpos_tags, spacy_ner_tags, triple, triple_tag[0]) if a != None]).most_common(1)[0][0]) 
        else:
            subject_person.append(0)    
    
    object_person = []
    for element in tokenized_triple[2]:
        if element in First_Pronouns:        
            object_person.append(1)
        elif element in Second_Pronouns:
            object_person.append(2)
        elif element in Third_Pronouns:
            object_person.append(3)
        elif len(stpos_tag_check(stpos_tags, spacy_ner_tags, triple, triple_tag[2])) != 0:
            object_person.append(Counter([a for a in stpos_tag_check(stpos_tags, spacy_ner_tags, triple, triple_tag[2]) if a != None]).most_common(1)[0][0]) 
        else:
            object_person.append(0)
    
    subject_person = run_object(subject_person) 
    object_person = run_object(object_person)

    return subject_person, object_person, triple['subjectSpan'][0], triple['objectSpan'][0]

            ############# all tenses labels##############
            ##########  Future Perfect Continuous --------> 1
            ##########        Future Perfect      --------> 2
            ##########       Future Continuous    --------> 3
            ##########          Future Simple     --------> 4
            ##########  present Perfect Continuous --------> 5
            ##########        present Perfect      --------> 6
            ##########       present Continuous    --------> 7
            ##########          present Simple     --------> 8            
            ##########  past Perfect Continuous --------> 9
            ##########        past Perfect      --------> 10
            ##########       past Continuous    --------> 11
            ##########          past Simple     --------> 12
            ##########          Undetectable      --------> 0
            #############################################

In [6]:
def tense_detector(stpos_tags, openie, sub_location, obj_location):
    all_tense = []
    for triple in openie:
        if sub_location == triple['subjectSpan'][0] and obj_location == triple['objectSpan'][0]:
            predicate_index = triple['relationSpan']
            only_stpos_tag = [v for k,v in stpos_tags[predicate_index[0]:predicate_index[1]+1]]

            if "MD" in [w for w in only_stpos_tag]:
                if set(["VB", "VBN", "VBG"]).issubset(set(only_stpos_tag)) == True: #Future Perfect Continuous
                    all_tense.append(1)
                    #print 'Future Perfect Continuous'
                    
                elif set(["VB", "VBN"]).issubset(set(only_stpos_tag)) == True: #Future Perfect 
                    all_tense.append(2)
                    #print 'Future Perfect'
                elif set(["VB", "VBG"]).issubset(set(only_stpos_tag)) == True: #Future Continuous
                    all_tense.append(3)
                    #print 'Future Continuous'
                else: #Future
                    all_tense.append(4)
                    #print 'Future'

            elif "VBD" in [w for w in only_stpos_tag]:
                if set(["VBN", "VBG"]).issubset(set(only_stpos_tag)) == True: #Past Perfect Continuous
                    all_tense.append(9)
                    #print 'Past Perfect Continuous'
                elif set(["VBN"]).issubset(set(only_stpos_tag)) == True: #Past Perfect 
                    all_tense.append(10)
                    #print 'Past Perfect'
                elif set(["VBG"]).issubset(set(only_stpos_tag)) == True: #Past Continuous
                    all_tense.append(11)
                    #print 'Past Continuous'
                else: #Past
                    all_tense.append(12)
                    #print 'Past'

            elif "VBP" or "VBZ" in [w for w in only_stpos_tag]:
                if set(["VBN", "VBG"]).issubset(set(only_stpos_tag)) == True: #present Perfect Continuous
                    all_tense.append(5)
                    #print 'present Perfect Continuous'
                elif set(["VBN"]).issubset(set(only_stpos_tag)) == True: #present Perfect 
                    all_tense.append(6)
                    #print 'present Perfect'
                elif set(["VBG"]).issubset(set(only_stpos_tag)) == True: #present Continuous
                    all_tense.append(7)
                    #print 'present Continuous'
                else:#present 
                    all_tense.append(8)
                    #print 'present'

            else:
                all_tense.append(0)
                #print 'undetectable'
            
    if len(Counter(all_tense)) > 1:
        all_tense = [max([a for a,b in Counter(all_tense).items() if a != 0])]
    else: 
        all_tense = [a for a,b in Counter(all_tense).items()]
    
    return all_tense

In [7]:
def tense_detector(stpos_tags, openie, sub_location, obj_location):
    all_tense = []
    for triple in openie:
        if sub_location == triple['subjectSpan'][0] and obj_location == triple['objectSpan'][0]:
            predicate_index = triple['relationSpan']
            only_stpos_tag = [v for v in stpos_tags[predicate_index[0]:predicate_index[-1]]]
                              
            if "MD" in [w for w in only_stpos_tag]: 
                if set(["VB", "VBN", "VBG"]).issubset(set(only_stpos_tag)) == True: #Future Perfect Continuous
                    all_tense.append(1) 
                elif set(["VB", "VBN"]).issubset(set(only_stpos_tag)) == True: #Future Perfect 
                    all_tense.append(2)
                elif set(["VB", "VBG"]).issubset(set(only_stpos_tag)) == True: #Future Continuous
                    all_tense.append(3)
                else: #Future
                    all_tense.append(4)
                              
            elif "VBP" or "VBZ" in [w for w in only_stpos_tag]:
                if set(["VBN", "VBG"]).issubset(set(only_stpos_tag)) == True: #present Perfect Continuous
                    all_tense.append(5)
                elif set(["VBN"]).issubset(set(only_stpos_tag)) == True: #present Perfect 
                    all_tense.append(6)
                elif set(["VBG"]).issubset(set(only_stpos_tag)) == True: #present Continuous
                    all_tense.append(7)
                else:#present 
                    all_tense.append(8)
                              
            elif "VBD" in [w for w in only_stpos_tag]:
                if set(["VBN", "VBG"]).issubset(set(only_stpos_tag)) == True: #Past Perfect Continuous
                    all_tense.append(9)
                elif set(["VBN"]).issubset(set(only_stpos_tag)) == True: #Past Perfect 
                    all_tense.append(10)
                elif set(["VBG"]).issubset(set(only_stpos_tag)) == True: #Past Continuous
                    all_tense.append(11)
                else: #Past
                    all_tense.append(12)       

            else:
                all_tense.append(0) # 'undetectable'
        else:
            continue
   
    if len(all_tense) > 0: 
        tense = min(all_tense)
    else: 
        tense = 0
    
    return tense

    

In [18]:
nlp = StanfordCoreNLP('http://localhost:9000')
spacy_ner = spacy.load("en_core_web_sm")

def triple_selection(input_file, output_file): # input_file is the raw dataset; 
                                               #output_file is used to stored the selected triple 
                                               #in forms of viewpoints and tense
    f1 = codecs.open(input_file, encoding='utf-8')# after executing obj.txt, sub.txt need to be processed by modifying open("MPQA/obj.txt") with open("MPQA/subj.txt")
    uni_lines = f1.readlines()
    sentence_number = -1
    all_data_sub_person = []
    all_data_obj_person = []
    all_data_tense = []
    all_data_sub_obj = []
    all_data_sub_tense = []
    all_data_tense_obj = []
    all_data_triple = []
    for line in uni_lines:    
        sentence_number += 1
        line = line.replace("\n", '')
        line = line.replace(")", '')
        line = line.replace("(", '')
        #print ("No.", sentence_number+1,"sentense is being processed.------>", line)
        try:
            res = nlp.annotate(line.lower(),
                                       properties={
                                           'annotators': 'depparse, parse, openie',
                                           'outputFormat': 'json',"triple.strict":"true"
                                       })

            parse = res["sentences"][0]["parse"]
            openie = res["sentences"][0]["openie"]

            stpos_tags = [a.tag_ for a in spacy_ner(line)]
            pos_tags = [a[1] for a in pos_tag(word_tokenize(line))]
            spacy_ner_tags = [a.ent_type_ for a in spacy_ner(line)]

        except:
            #print("No.",sentence_number+1,"sentence can't be analyzed by corenlp")
            continue

        sum_subs = []
        sum_subs_location = []
        sum_objs_location = []
        sum_objs = []
        sum_verbs= []
        sub_obj = []
        sub_tense = []
        tense_obj = []
        sub_tense_obj = []
        try:
            for triple in openie:
                sub_person, obj_person, sub_location, obj_location = persons_check(triple, stpos_tags, spacy_ner_tags)

        # experiment 1 : test subject only
                sum_subs_location.append(sub_location)
                sum_objs_location.append(obj_location)
                sum_subs.append(sub_person)

        # experiment 2 : test object only
                sum_objs.append(obj_person) 

        except:
            #print("subject or coresponding object-list is failed to be extracted")
            continue

        try:
            # experiment 1 : test subject only 
            external_sub = sum_subs[sum_subs_location.index(min(sum_subs_location))] 
            # experiment 2 : test object only

            select_sub_o_location = [triple['objectSpan'][0] for triple in openie if min(sum_subs_location) == triple['subjectSpan'][0]]

            selected_objs_person = sum_objs[sum_objs_location.index(max(select_sub_o_location))]   

            tense = tense_detector(pos_tags, openie, min(sum_subs_location), max(select_sub_o_location))


            all_data_triple.append([external_sub]+ [tense]+ [selected_objs_person])
        except:
            #print("No.",sentence_number+1,"sentence fails to be processed")
            continue

        #print('*********************************************************************************************************************')
        if len([external_sub]+ [tense]+[selected_objs_person]) > 3:
            print ('no triples can be extracted from the sentence:', line)
    f.close()
    outputs = open(output_file, 'w')
    outputs.write(str(all_data_triple))
    outputs.close()
    #return all_data_triple

In [29]:
# save the results to txt files
triple_selection("MPQA/obj-samples.txt", 'temp_outputs/obj_triples.txt') # for objective dataset
triple_selection("MPQA/subj-samples.txt", 'temp_outputs/subj_triples.txt')# for subjective dataset

In [30]:
def dataprocessing(file_location):

    fd = open(file_location, "r")
    all_data = (fd.readlines())[0][2:-2].split('], [')

    all_subs = []
    all_tenses = []
    all_objs = []
    all_subs_tenses = []
    all_subs_objs = []
    all_tenses_objs = []
    
    for tem_triple in (all_data):  
        
        tem_triple_ = tem_triple.split(', ')
        all_subs.append((tem_triple_[0]))
        all_tenses.append((tem_triple_[1]))  
        all_objs.append((tem_triple_[-1]))
        all_subs_tenses.append((tem_triple_[0], tem_triple_[1]))
        all_subs_objs.append((tem_triple_[0],tem_triple_[-1]))
        all_tenses_objs.append((tem_triple_[1], tem_triple_[-1]))

    return all_subs_tenses, all_subs_objs, all_tenses_objs, all_data

In [34]:
# subjective dataset
S_all_subs_tenses, S_all_subs_objs, S_all_tenses_objs, S_data = dataprocessing('temp_outputs/subj_triples.txt')

#  objective dataset
O_all_subs_tenses, O_all_subs_objs, O_all_tenses_objs, O_data = dataprocessing('temp_outputs/obj_triples.txt')

# print('S_all_subs_tenses', Counter(S_all_subs_tenses).items()) 
# print('O_all_subs_tenses', Counter(O_all_subs_tenses).items()) 
# print('****************************************************************')
# print('S_all_subs_objs', Counter(S_all_subs_objs).items()) 
# print('O_all_subs_objs', Counter(O_all_subs_objs).items()) 
# print('****************************************************************')
# print('S_all_tenses_objs', Counter(S_all_tenses_objs).items()) 
# print('O_all_tenses_objs', Counter(O_all_tenses_objs).items()) 
# print('****************************************************************')
# print('S_data', Counter(S_data).items()) 
# print('O_data', Counter(O_data).items()) 
# print('****************************************************************')
# print(S_data,len(S_data))
# print(O_data,len(O_data))

In [94]:
#unique tuple - s v
countered_S = Counter(S_all_subs_tenses).items()
countered_O = Counter(O_all_subs_tenses).items()

sub_triple_100 = [(a, b) for a, b in countered_S if a not in O_all_subs_tenses]
print 'S for sv', sub_triple_100    

obj_triple_100 = [(a, b) for a, b in countered_O if a not in S_all_subs_tenses]
print 'O for sv', obj_triple_100 

#unique tuple - v o
countered_S = Counter(S_all_tenses_objs).items()
countered_O = Counter(O_all_tenses_objs).items()

sub_triple_100 = [(a, b) for a, b in countered_S if a not in O_all_tenses_objs]
print 'S for vo', sub_triple_100    

obj_triple_100 = [(a, b) for a, b in countered_O if a not in S_all_tenses_objs]
print 'O for vo', obj_triple_100 
#unique tuple - s o
countered_S = Counter(S_all_subs_objs).items()
countered_O = Counter(O_all_subs_objs).items()

sub_triple_100 = [(a, b) for a, b in countered_S if a not in O_all_subs_objs]
print 'S for so', sub_triple_100    

obj_triple_100 = [(a, b) for a, b in countered_O if a not in S_all_subs_objs]
print 'O for so', sub_triple_100   

# unique triples
countered_S = Counter(S_data).items()
countered_O = Counter(O_data).items()

sub_triple_100 = [(a, b) for a, b in countered_S if a not in O_data]
print 's triple', sub_triple_100     

obj_triple_100 = [(a, b) for a, b in countered_O if a not in S_data]
print 'o triple', obj_triple_100  


S for sv [(('3', '11'), 1), (('0', '2'), 1), (('4', '4'), 2), (('0', '3'), 1), (('1', '12'), 11), (('1', '11'), 1), (('1', '10'), 3), (('1', '7'), 8)]
O for sv [(('3', '10'), 4), (('2', '11'), 1)]
S for vo [(('3', '0'), 1), (('4', '1'), 1), (('2', '0'), 1), (('12', '2'), 1)]
O for vo [(('10', '4'), 3), (('12', '4'), 8), (('11', '4'), 1), (('6', '1'), 1), (('11', '3'), 2), (('10', '3'), 2)]
S for so [(('4', '2'), 1), (('2', '1'), 1), (('2', '4'), 2)]
O for so [(('4', '2'), 1), (('2', '1'), 1), (('2', '4'), 2)]
s triple [('0, 4, 1', 1), ('2, 7, 3', 1), ('2, 6, 3', 1), ('1, 12, 3', 3), ('1, 12, 0', 8), ('4, 4, 0', 2), ('0, 3, 0', 1), ('0, 12, 1', 2), ('0, 12, 2', 1), ('3, 11, 0', 1), ('0, 2, 0', 1), ('2, 12, 3', 2), ('4, 7, 2', 1), ('1, 10, 0', 3), ('1, 7, 3', 1), ('2, 6, 4', 1), ('1, 11, 0', 1), ('2, 4, 3', 2), ('2, 4, 2', 2), ('2, 8, 4', 1), ('1, 7, 0', 7), ('2, 8, 1', 1)]
o triple [('0, 10, 4', 2), ('4, 12, 4', 1), ('0, 11, 4', 1), ('0, 6, 1', 1), ('4, 6, 3', 1), ('0, 11, 3', 2), ('4, 

In [99]:
pair_ST = [S_all_subs_tenses, O_all_subs_tenses]   
pair_SO = [S_all_subs_objs, O_all_subs_objs]
pair_TO = [S_all_tenses_objs, O_all_tenses_objs]
pair_STO = [S_data, O_data]
    
countered_S = Counter(pair_STO[0]).items()
countered_O = Counter(pair_STO[1]).items()

############## possibility calculations ###############        ######(1,1,1) ---> sub for 0 times
all_S_percentages = [] #### for subjective dataset         ######        ---> obj for 3 times  
for item_s, num_s in countered_S:
    try:
        num_o = [b for a, b in countered_O if a==item_s][0]
        percentage_S = float(num_s)/(num_s+num_o) #
        all_S_percentages.append((item_s,percentage_S, num_s, num_o))
    except:
        continue


all_O_percentages = [] #### for objective dataset
for item_o, num_o in countered_O:
    try:
        num_s = [b for a, b in countered_S if a==item_o][0]
        percentage_O = float(num_o)/(num_s+num_o)
        all_O_percentages.append((item_o,percentage_O, num_s, num_o))
    except:
        continue
#print all_S_percentages
#print all_O_percentages

thresholds = [0.7, 0.725, 0.75, 0.775, 0.8, 0.825, 0.85, 0.875, 0.9, 0.925, 1]
for threshold in thresholds:
    subjective_patterns = []
    objective_patterns = []
    for item_s, percentage_S, num_s, num_o in all_S_percentages:

        if percentage_S >= threshold and percentage_S < thresholds[(thresholds.index(threshold)+1)]:  
              # if possibility of being a subjective one is larger than threshold, 
            subjective_patterns.append(((item_s), num_s, num_o, percentage_S,))    # the current item will be selected as a subjective pattern
   

    for item_s, percentage_O, num_s, num_o in all_O_percentages:    
        if percentage_O >= threshold and percentage_O < thresholds[(thresholds.index(threshold)+1)]:  
              # if possibility of being a subjective one is larger than threshold, 
            objective_patterns.append(((item_s), num_s, num_o, percentage_O)) 
    #             if percentage_S <= 0.5:  
    #                 if item_s not in objective_patterns:      # if possibility of being a subjective one is smaller than threshold,
    #                     objective_patterns.append(item_s)     # the current item will be selected as a objective pattern   

    print 'when threshold is', threshold, ' subjective patterns are', subjective_patterns
    print 'when threshold is', threshold, 'objective patterns are', objective_patterns

when threshold is 0.7  subjective patterns are [('1, 6, 0', 7, 3, 0.7), ('2, 8, 3', 7, 3, 0.7)]
when threshold is 0.7 objective patterns are [('0, 6, 0', 110, 282, 0.7193877551020408), ('0, 8, 3', 152, 364, 0.7054263565891473)]
when threshold is 0.725  subjective patterns are [('0, 8, 1', 28, 10, 0.7368421052631579)]
when threshold is 0.725 objective patterns are [('3, 7, 0', 22, 64, 0.7441860465116279), ('3, 8, 4', 6, 17, 0.7391304347826086)]
when threshold is 0.75  subjective patterns are [('1, 8, 4', 3, 1, 0.75)]
when threshold is 0.75 objective patterns are [('4, 8, 3', 2, 6, 0.75), ('4, 8, 0', 17, 52, 0.7536231884057971), ('0, 10, 0', 6, 20, 0.7692307692307693)]
when threshold is 0.775  subjective patterns are [('1, 4, 0', 7, 2, 0.7777777777777778), ('3, 8, 2', 7, 2, 0.7777777777777778)]
when threshold is 0.775 objective patterns are [('4, 6, 0', 2, 7, 0.7777777777777778), ('0, 8, 4', 20, 78, 0.7959183673469388)]
when threshold is 0.8  subjective patterns are [('3, 8, 1', 9, 2, 0.

In [96]:
########cross fold testing##########
import numpy as np
import random
import prettytable as pt

for threshold in [0.7, 0.725, 0.75, 0.775, 0.8, 0.825, 0.85, 0.875, 0.9, 0.925]:
    tb = pt.PrettyTable()
    subjective_result = []
    objective_result = []
    precision_S_all = []
    precision_O_all = []
    recall_S_all = []
    recall_O_all = []
    fscore_S_all = []
    fscore_O_all = []
    for b in range(100):
        sub_accuracy = 0
        obj_accuracy = 0
        count_sub = 0
        count_obj = 0
        #print len(S_data), len(O_data)
        for a in range(10):
            S_data_sorting = sorted(S_data, key=lambda k: random.random()) # randomly sort data points
            O_data_sorting = sorted(O_data, key=lambda k: random.random())
            splitted_S_data = [S_data_sorting[i:i + (len(S_data)/10)+1] for i in range(0, len(S_data_sorting), (len(S_data)/10)+1)] #np.array(np.array_split(S_data_sorting,10,axis=0))
            splitted_O_data = [O_data_sorting[i:i + (len(O_data)/10)+1] for i in range(0, len(O_data_sorting), (len(O_data)/10)+1)] #np.array(np.array_split(O_data_sorting,10,axis=0))

            testing_S_data = splitted_S_data.pop(a)
            testing_O_data = splitted_O_data.pop(a)
            _training_S_data = splitted_S_data
            _training_O_data = splitted_O_data

            training_S_data = []
            training_O_data = []  
            for a in _training_S_data:
                training_S_data.extend(a)  
            for a in _training_O_data:
                training_O_data.extend(a)

            countered_S = Counter(training_S_data).items()
            countered_O = Counter(training_O_data).items()

        ############## possibility calculations ###############        ######(1,1,1) ---> sub for 0 times
            all_S_percentages = [] #### for subjective dataset         ######        ---> obj for 3 times  
            for item_s, num_s in countered_S:
                try:
                    num_o = [b for a, b in countered_O if a==item_s][0]
                    percentage_S = float(num_s)/(num_s+num_o) #
                    all_S_percentages.append((item_s,percentage_S))
                except:
                    continue


            all_O_percentages = [] #### for objective dataset
            for item_o, num_o in countered_O:
                try:
                    num_s = [b for a, b in countered_S if a==item_o][0]
                    percentage_O = float(num_o)/(num_s+num_o)
                    all_O_percentages.append((item_o,percentage_O))
                except:
                    continue


    #         print all_S_percentages
    #         print all_O_percentages

            subjective_patterns = []
            objective_patterns = []
            for item_s, percentage_S in all_S_percentages:

                if percentage_S >= threshold:  
                    if item_s not in subjective_patterns:     # if possibility of being a subjective one is larger than threshold, 
                        subjective_patterns.append(item_s)    # the current item will be selected as a subjective pattern

    #             if percentage_S <= 0.5:  
    #                 if item_s not in objective_patterns:      # if possibility of being a subjective one is smaller than threshold,
    #                     objective_patterns.append(item_s)     # the current item will be selected as a objective pattern   

            for item_o, percentage_O in all_O_percentages:

                if percentage_O >= threshold:  
                    if item_o not in objective_patterns:      # if possibility of being a objective one is larger than threshold,
                        objective_patterns.append(item_o)     # the current item will be selected as a objective pattern 

    #             if percentage_O <= 0.5: 
    #                 if item_o not in subjective_patterns:      # if possibility of being a objective one is smaller than threshold,
    #                     subjective_patterns.append(item_o)    # the current item will be selected as a subjective pattern  

            Sub_positive = 0
            Obj_positive = 0
            Sub_negative = 0
            Obj_negative = 0
            for a in testing_S_data:
                if a in subjective_patterns:
                    Sub_positive += 1
                elif a in objective_patterns:
                    Sub_negative += 1
                else:
                    continue
            for a in testing_O_data:
                if a in subjective_patterns:
                    Obj_negative += 1
                elif a in objective_patterns:
                    Obj_positive += 1
                else:
                    continue


            try:
                accuracy = (float(Sub_positive + Obj_positive))/(Sub_positive + Sub_negative + Obj_positive + Obj_negative)
                precision_S = (float(Sub_positive))/(Sub_positive + Obj_negative)
                precision_O = (float(Obj_positive))/(Obj_positive + Sub_negative)
                Recall_S = (float(Sub_positive))/(Sub_positive + Sub_negative)
                Recall_O = (float(Obj_positive))/(Obj_positive + Obj_negative)
                Fscore_S = (2*Recall_S*precision_S)/(Recall_S+precision_S)
                Fscore_O = (2*Recall_O*precision_O)/(Recall_O+precision_O)
                count_sub += 1
            except:
                continue
    #         print accuracy, Sub_positive, Obj_positive, Sub_negative, Obj_negative    
            subjective_result.append(accuracy)
            precision_S_all.append(precision_S)
            precision_O_all.append(precision_O)
            recall_S_all.append(Recall_S)
            recall_O_all.append(Recall_O)
            fscore_S_all.append(Fscore_S)
            fscore_O_all.append(Fscore_O)
#             print ('all_S_percentages', all_S_percentages)
#             print ('all_O_percentages', all_O_percentages)
#             break
#     #     subjective_result.append(sub_accuracy/count_sub)
#     #     objective_result.append(obj_accuracy/count_obj)
#         break
    print 'when the threshold = ', threshold
#     print  subjective_result
    tb.field_names = ["Measure", "Mean", "variance", "standard deviation"]
    #print ['accuracy',round(np.mean(subjective_result),5), round(np.var(subjective_result),5), round(np.std(subjective_result),2)]
    tb.add_row(['accuracy',round(np.mean(subjective_result),5), round(np.var(subjective_result),5), round(np.std(subjective_result),5)])
    tb.add_row(['precision_S_all',round(np.mean(precision_S_all),5), round(np.var(precision_S_all),5), round(np.std(precision_S_all),5)])
    tb.add_row(['precision_O_all',round(np.mean(precision_O_all),5), round(np.var(precision_O_all),5),round(np.std(precision_O_all),5)])
    tb.add_row(['recall_S_all',round(np.mean(recall_S_all),5), round(np.var(recall_S_all),5), round(np.std(recall_S_all),5)])
    tb.add_row(['recall_O_all',round(np.mean(recall_O_all),5), round(np.var(recall_O_all),5), round(np.std(recall_O_all),5)])
    tb.add_row(['fscore_S_all',round(np.mean(fscore_S_all),5), round(np.var(fscore_S_all),5), round(np.std(fscore_S_all),5)])
    tb.add_row(['fscore_O_all',round(np.mean(fscore_O_all),5), round(np.var(fscore_O_all),5), round(np.std(fscore_O_all),5)])
    tb.add_row(['average precision', round(float(np.mean(precision_S_all)+np.mean(precision_O_all))/2,5), round(float(np.var(precision_S_all)+np.var(precision_O_all))/2,5), round(float(np.std(precision_S_all)+np.std(precision_O_all))/2,5)])
    tb.add_row(['average recall', round(float(np.mean(recall_S_all)+np.mean(recall_O_all))/2,5), round(float(np.var(recall_S_all)+np.var(recall_O_all))/2,5), round(float(np.std(recall_S_all)+np.std(recall_O_all))/2,5)])
    tb.add_row(['average fscore', round(float(np.mean(fscore_S_all)+np.mean(fscore_O_all))/2,5), round(float(np.var(fscore_S_all)+np.var(fscore_O_all))/2,5), round(float(np.std(fscore_S_all)+np.std(fscore_O_all))/2,5)])
    print (tb)


#print (subjective_result)
# print (objective_result)



when the threshold =  0.7
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.76182 | 0.00077  |      0.02776       |
|  precision_S_all  | 0.76878 | 0.00992  |      0.09959       |
|  precision_O_all  | 0.76158 | 0.00084  |      0.02892       |
|    recall_S_all   | 0.24793 | 0.00493  |      0.07021       |
|    recall_O_all   | 0.96938 | 0.00026  |      0.01608       |
|    fscore_S_all   | 0.36974 | 0.00672  |      0.08195       |
|    fscore_O_all   | 0.85263 | 0.00035  |       0.0188       |
| average precision | 0.76518 | 0.00538  |      0.06425       |
|   average recall  | 0.60866 | 0.00259  |      0.04315       |
|   average fscore  | 0.61119 | 0.00353  |      0.05037       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.725
+-------------------+---------+----------+--------

when the threshold =  0.925
+-------------------+------+----------+--------------------+
|      Measure      | Mean | variance | standard deviation |
+-------------------+------+----------+--------------------+
|      accuracy     | nan  |   nan    |        nan         |
|  precision_S_all  | nan  |   nan    |        nan         |
|  precision_O_all  | nan  |   nan    |        nan         |
|    recall_S_all   | nan  |   nan    |        nan         |
|    recall_O_all   | nan  |   nan    |        nan         |
|    fscore_S_all   | nan  |   nan    |        nan         |
|    fscore_O_all   | nan  |   nan    |        nan         |
| average precision | nan  |   nan    |        nan         |
|   average recall  | nan  |   nan    |        nan         |
|   average fscore  | nan  |   nan    |        nan         |
+-------------------+------+----------+--------------------+


when the threshold =  0.7
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     |  0.7646 | 0.00082  |      0.02864       |
|  precision_S_all  | 0.77369 | 0.00985  |      0.09923       |
|  precision_O_all  |  0.764  | 0.00084  |      0.02898       |
|    recall_S_all   | 0.25333 | 0.00515  |      0.07176       |
|    recall_O_all   | 0.96992 | 0.00024  |      0.01553       |
|    fscore_S_all   | 0.37666 | 0.00706  |      0.08404       |
|    fscore_O_all   | 0.85439 | 0.00037  |      0.01927       |
| average precision | 0.76885 | 0.00534  |      0.06411       |
|   average recall  | 0.61163 |  0.0027  |      0.04365       |
|   average fscore  | 0.61552 | 0.00372  |      0.05165       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.725
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.79999 | 0.00173  |      0.04157       |
|  precision_S_all  | 0.76385 | 0.01117  |       0.1057       |
|  precision_O_all  | 0.80737 | 0.00201  |      0.04482       |
|    recall_S_all   | 0.40509 | 0.01434  |      0.11974       |
|    recall_O_all   | 0.95167 | 0.00069  |      0.02626       |
|    fscore_S_all   | 0.51882 | 0.01309  |      0.11443       |
|    fscore_O_all   | 0.87271 | 0.00077  |      0.02781       |
| average precision | 0.78561 | 0.00659  |      0.07526       |
|   average recall  | 0.67838 | 0.00751  |       0.073        |
|   average fscore  | 0.69576 | 0.00693  |      0.07112       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.75
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.81192 | 0.00148  |      0.03852       |
|  precision_S_all  | 0.77353 | 0.01274  |      0.11289       |
|  precision_O_all  | 0.82009 | 0.00152  |      0.03904       |
|    recall_S_all   | 0.42838 | 0.01059  |       0.1029       |
|    recall_O_all   | 0.95182 | 0.00081  |      0.02848       |
|    fscore_S_all   | 0.54329 | 0.00933  |       0.0966       |
|    fscore_O_all   | 0.88044 | 0.00073  |      0.02702       |
| average precision | 0.79681 | 0.00713  |      0.07596       |
|   average recall  |  0.6901 |  0.0057  |      0.06569       |
|   average fscore  | 0.71186 | 0.00503  |      0.06181       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.775
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.82339 |  0.0018  |      0.04237       |
|  precision_S_all  | 0.79912 | 0.01366  |      0.11688       |
|  precision_O_all  | 0.82866 | 0.00205  |      0.04529       |
|    recall_S_all   | 0.44774 | 0.01352  |      0.11628       |
|    recall_O_all   | 0.95849 | 0.00075  |      0.02737       |
|    fscore_S_all   | 0.56459 | 0.01142  |      0.10688       |
|    fscore_O_all   | 0.88803 | 0.00084  |      0.02906       |
| average precision | 0.81389 | 0.00786  |      0.08109       |
|   average recall  | 0.70311 | 0.00713  |      0.07182       |
|   average fscore  | 0.72631 | 0.00613  |      0.06797       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.8
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.82406 | 0.00208  |      0.04561       |
|  precision_S_all  | 0.80763 | 0.01574  |      0.12545       |
|  precision_O_all  | 0.82896 | 0.00247  |      0.04974       |
|    recall_S_all   |  0.4726 | 0.01868  |      0.13669       |
|    recall_O_all   | 0.95499 | 0.00109  |      0.03294       |
|    fscore_S_all   | 0.58312 | 0.01466  |      0.12107       |
|    fscore_O_all   | 0.88647 | 0.00104  |      0.03231       |
| average precision | 0.81829 | 0.00911  |      0.08759       |
|   average recall  | 0.71379 | 0.00988  |      0.08482       |
|   average fscore  |  0.7348 | 0.00785  |      0.07669       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.825
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.83697 | 0.00278  |      0.05277       |
|  precision_S_all  | 0.76507 | 0.02698  |      0.16427       |
|  precision_O_all  | 0.85432 | 0.00317  |       0.0563       |
|    recall_S_all   | 0.45673 | 0.03141  |      0.17724       |
|    recall_O_all   | 0.94967 | 0.00179  |      0.04227       |
|    fscore_S_all   | 0.54541 |  0.0224  |      0.14968       |
|    fscore_O_all   | 0.89801 | 0.00135  |      0.03674       |
| average precision | 0.80969 | 0.01508  |      0.11028       |
|   average recall  |  0.7032 |  0.0166  |      0.10976       |
|   average fscore  | 0.72171 | 0.01188  |      0.09321       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.85
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.85808 | 0.00409  |      0.06393       |
|  precision_S_all  | 0.67052 | 0.04832  |      0.21981       |
|  precision_O_all  | 0.88997 | 0.00423  |      0.06505       |
|    recall_S_all   | 0.45108 | 0.05378  |       0.2319       |
|    recall_O_all   | 0.94601 | 0.00221  |      0.04697       |
|    fscore_S_all   | 0.49321 | 0.02757  |      0.16604       |
|    fscore_O_all   | 0.91535 | 0.00177  |      0.04209       |
| average precision | 0.78024 | 0.02627  |      0.14243       |
|   average recall  | 0.69855 | 0.02799  |      0.13944       |
|   average fscore  | 0.70428 | 0.01467  |      0.10406       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.875
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.86316 | 0.00344  |      0.05866       |
|  precision_S_all  | 0.51537 | 0.01807  |      0.13444       |
|  precision_O_all  | 0.91032 |  0.0033  |      0.05745       |
|    recall_S_all   | 0.48395 | 0.05388  |      0.23212       |
|    recall_O_all   | 0.93169 |  0.0014  |      0.03742       |
|    fscore_S_all   | 0.47338 |  0.0229  |      0.15132       |
|    fscore_O_all   | 0.91975 | 0.00142  |      0.03766       |
| average precision | 0.71284 | 0.01069  |      0.09595       |
|   average recall  | 0.70782 | 0.02764  |      0.13477       |
|   average fscore  | 0.69656 | 0.01216  |      0.09449       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.9
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.87568 | 0.00299  |      0.05472       |
|  precision_S_all  | 0.57292 | 0.01319  |      0.11483       |
|  precision_O_all  |  0.9172 | 0.00314  |      0.05604       |
|    recall_S_all   | 0.52982 |  0.0568  |      0.23832       |
|    recall_O_all   | 0.93902 | 0.00217  |      0.04655       |
|    fscore_S_all   |  0.5276 | 0.02343  |      0.15307       |
|    fscore_O_all   | 0.92628 | 0.00167  |      0.04084       |
| average precision | 0.74506 | 0.00816  |      0.08544       |
|   average recall  | 0.73442 | 0.02948  |      0.14243       |
|   average fscore  | 0.72694 | 0.01255  |      0.09696       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.925
+-------------------+------+----------+--------------------+
|      Measure      | Mean | variance | standard deviation |
+-------------------+------+----------+--------------------+
|      accuracy     | nan  |   nan    |        nan         |
|  precision_S_all  | nan  |   nan    |        nan         |
|  precision_O_all  | nan  |   nan    |        nan         |
|    recall_S_all   | nan  |   nan    |        nan         |
|    recall_O_all   | nan  |   nan    |        nan         |
|    fscore_S_all   | nan  |   nan    |        nan         |
|    fscore_O_all   | nan  |   nan    |        nan         |
| average precision | nan  |   nan    |        nan         |
|   average recall  | nan  |   nan    |        nan         |
|   average fscore  | nan  |   nan    |        nan         |
+-------------------+------+----------+--------------------+

before modifying the experiment

when the threshold =  0.7
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.79116 | 0.00151  |      0.03887       |
|  precision_S_all  | 0.78521 | 0.00993  |      0.09966       |
|  precision_O_all  | 0.79474 | 0.00163  |       0.0404       |
|    recall_S_all   |  0.4562 | 0.00884  |      0.09399       |
|    recall_O_all   | 0.94041 | 0.00125  |      0.03541       |
|    fscore_S_all   | 0.56982 | 0.00682  |      0.08257       |
|    fscore_O_all   | 0.86068 | 0.00086  |      0.02938       |
| average precision | 0.78998 | 0.00578  |      0.07003       |
|   average recall  |  0.6983 | 0.00504  |       0.0647       |
|   average fscore  | 0.71525 | 0.00384  |      0.05598       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.725
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.79595 | 0.00153  |      0.03913       |
|  precision_S_all  | 0.80506 | 0.01137  |      0.10664       |
|  precision_O_all  | 0.79594 | 0.00167  |      0.04083       |
|    recall_S_all   |  0.4179 |  0.0108  |      0.10392       |
|    recall_O_all   | 0.95463 |  0.001   |       0.0317       |
|    fscore_S_all   | 0.54141 | 0.00954  |      0.09769       |
|    fscore_O_all   | 0.86733 | 0.00081  |      0.02847       |
| average precision |  0.8005 | 0.00652  |      0.07374       |
|   average recall  | 0.68626 |  0.0059  |      0.06781       |
|   average fscore  | 0.70437 | 0.00518  |      0.06308       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.75
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.81992 | 0.00216  |      0.04646       |
|  precision_S_all  | 0.82353 | 0.01244  |      0.11153       |
|  precision_O_all  | 0.82092 | 0.00272  |      0.05211       |
|    recall_S_all   | 0.50497 | 0.01616  |      0.12712       |
|    recall_O_all   | 0.95243 | 0.00111  |      0.03327       |
|    fscore_S_all   | 0.61531 | 0.01182  |      0.10871       |
|    fscore_O_all   | 0.88061 | 0.00111  |      0.03325       |
| average precision | 0.82223 | 0.00758  |      0.08182       |
|   average recall  |  0.7287 | 0.00863  |       0.0802       |
|   average fscore  | 0.74796 | 0.00646  |      0.07098       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.775
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.83345 | 0.00206  |      0.04542       |
|  precision_S_all  | 0.84618 | 0.01247  |      0.11167       |
|  precision_O_all  |  0.8324 |  0.0023  |      0.04797       |
|    recall_S_all   | 0.51211 | 0.01482  |      0.12172       |
|    recall_O_all   | 0.96058 | 0.00105  |      0.03233       |
|    fscore_S_all   | 0.62838 | 0.01162  |       0.1078       |
|    fscore_O_all   | 0.89101 | 0.00106  |      0.03253       |
| average precision | 0.83929 | 0.00739  |      0.07982       |
|   average recall  | 0.73634 | 0.00793  |      0.07703       |
|   average fscore  |  0.7597 | 0.00634  |      0.07016       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.8
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.83183 | 0.00203  |      0.04505       |
|  precision_S_all  | 0.84887 |  0.0143  |       0.1196       |
|  precision_O_all  | 0.82976 | 0.00238  |      0.04879       |
|    recall_S_all   | 0.49641 | 0.01654  |      0.12862       |
|    recall_O_all   | 0.96287 | 0.00103  |      0.03215       |
|    fscore_S_all   | 0.61528 |  0.0134  |      0.11576       |
|    fscore_O_all   | 0.89039 | 0.00104  |       0.0323       |
| average precision | 0.83932 | 0.00834  |       0.0842       |
|   average recall  | 0.72964 | 0.00879  |      0.08039       |
|   average fscore  | 0.75283 | 0.00722  |      0.07403       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.825
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.82554 | 0.00269  |      0.05191       |
|  precision_S_all  | 0.87793 | 0.01288  |      0.11351       |
|  precision_O_all  | 0.81613 |  0.0033  |       0.0574       |
|    recall_S_all   | 0.53409 | 0.02363  |      0.15371       |
|    recall_O_all   |  0.9594 | 0.00179  |      0.04235       |
|    fscore_S_all   | 0.64848 | 0.01672  |      0.12929       |
|    fscore_O_all   | 0.88052 | 0.00151  |      0.03887       |
| average precision | 0.84703 | 0.00809  |      0.08546       |
|   average recall  | 0.74675 | 0.01271  |      0.09803       |
|   average fscore  |  0.7645 | 0.00911  |      0.08408       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.85
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.86181 | 0.00453  |       0.0673       |
|  precision_S_all  | 0.92759 | 0.01254  |      0.11199       |
|  precision_O_all  | 0.84418 | 0.00659  |      0.08119       |
|    recall_S_all   | 0.64902 | 0.03265  |      0.18069       |
|    recall_O_all   | 0.97158 | 0.00201  |       0.0448       |
|    fscore_S_all   | 0.74722 | 0.02015  |      0.14194       |
|    fscore_O_all   | 0.90074 | 0.00261  |       0.0511       |
| average precision | 0.88589 | 0.00957  |      0.09659       |
|   average recall  |  0.8103 | 0.01733  |      0.11275       |
|   average fscore  | 0.82398 | 0.01138  |      0.09652       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.875
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.89432 |  0.008   |      0.08946       |
|  precision_S_all  | 0.94414 | 0.01059  |      0.10293       |
|  precision_O_all  | 0.87098 | 0.01659  |      0.12879       |
|    recall_S_all   | 0.81061 | 0.03788  |      0.19463       |
|    recall_O_all   | 0.95297 | 0.00945  |       0.0972       |
|    fscore_S_all   |  0.8565 | 0.01907  |       0.1381       |
|    fscore_O_all   |  0.9023 | 0.00842  |      0.09177       |
| average precision | 0.90756 | 0.01359  |      0.11586       |
|   average recall  | 0.88179 | 0.02366  |      0.14591       |
|   average fscore  |  0.8794 | 0.01375  |      0.11493       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.9
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.91035 | 0.00986  |      0.09927       |
|  precision_S_all  | 0.95383 | 0.01062  |      0.10307       |
|  precision_O_all  | 0.88452 | 0.02476  |      0.15735       |
|    recall_S_all   | 0.87896 | 0.02634  |       0.1623       |
|    recall_O_all   | 0.94362 | 0.01633  |      0.12779       |
|    fscore_S_all   | 0.90309 | 0.01384  |      0.11764       |
|    fscore_O_all   | 0.89903 | 0.01496  |       0.1223       |
| average precision | 0.91917 | 0.01769  |      0.13021       |
|   average recall  | 0.91129 | 0.02134  |      0.14505       |
|   average fscore  | 0.90106 |  0.0144  |      0.11997       |
+-------------------+---------+----------+--------------------+
when the threshold =  0.925
+-------------------+---------+----------+--------------------+
|      Measure      |   Mean  | variance | standard deviation |
+-------------------+---------+----------+--------------------+
|      accuracy     | 0.79571 | 0.00679  |      0.08241       |
|  precision_S_all  |  0.9534 | 0.01175  |      0.10839       |
|  precision_O_all  | 0.64954 | 0.01183  |      0.10875       |
|    recall_S_all   | 0.69516 | 0.01881  |      0.13716       |
|    recall_O_all   | 0.95254 | 0.01166  |      0.10798       |
|    fscore_S_all   | 0.79508 |  0.0137  |      0.11706       |
|    fscore_O_all   | 0.76398 | 0.00905  |      0.09516       |
| average precision | 0.80147 | 0.01179  |      0.10857       |
|   average recall  | 0.82385 | 0.01524  |      0.12257       |
|   average fscore  | 0.77953 | 0.01138  |      0.10611       |
+-------------------+---------+----------+--------------------+

In [51]:
######## cross fold testing##########
import math
from sklearn import svm
import numpy as np
import re
import random

np_S_data = [] #transform string to numpy array for subjective data
for a in S_data:

    a = [int(b) for b in a.split(', ')]
    if len(a) != 3:
        del a[2]
    if 8 in a:
        pass
    np_S_data.append(np.array(a))
    
np_S_data = np.array(np_S_data)
np_O_data = [] #transform string to numpy array for objective data

for a in O_data:
    a = [int(b) for b in a.split(', ')]
    if len(a) != 3:
        del a[2]
    if 8 in a:
        pass
    np_O_data.append(np.array(a))
np_O_data = np.array(np_O_data)

len_S_data = int(len(np_S_data))
len_O_data = int(len(np_O_data))

splitted_S_data = np.array(np.array_split(np_S_data,10,axis=0))
splitted_O_data = np.array(np.array_split(np_O_data,10,axis=0))

SUM_accuracy = 0
SUM_Precision =  0
SUM_recall = 0
SUM_Fscore = 0

#write_csv(path_write, [])

for loop_num in range(10):   
    splitted_S_data = sorted(splitted_S_data, key=lambda k: random.random())
    for fold in range(2):
        
        testing_S_data = splitted_S_data[fold]
        testing_O_data = splitted_O_data[fold]
        merged_testing_data = np.vstack((testing_S_data, testing_O_data))
#         print ('test', testing_S_data)
        label_S_testing = list(np.zeros(testing_S_data.shape[0], dtype = np.int))
        label_O_testing = list(np.ones(testing_O_data.shape[0], dtype = np.int))
        merged_testing_label = label_S_testing + label_O_testing
        training_S_data = None

        for a in (np.delete(splitted_S_data, fold)): #concatenate O training data
            if training_S_data is None:
                training_S_data = a
            else:
                training_S_data = np.vstack((training_S_data, a))
        training_O_data = None
        for a in (np.delete(splitted_O_data, fold)): #concatenate S training data
            if training_O_data is None:
                training_O_data = a
            else:
                training_O_data = np.vstack((training_O_data, a)) 

        label_S_training = list(np.zeros(training_S_data.shape[0], dtype = np.int))
        label_O_training = list(np.ones(training_O_data.shape[0], dtype = np.int))
        clf = svm.SVC()
        X = np.vstack((training_S_data, training_O_data))    
        y = label_S_training + label_O_training
        clf.fit(X, y)

    #     print 'Lengthes of training S and O data are', len(label_S_training), len(label_O_training)

    #     print 'Lengthes of testing S and O data are',len(testing_S_data), len(testing_O_data)
    #     #print X
        result = []
        for testing_item in merged_testing_data:
            #print testing_item
            result.append(clf.predict([testing_item])[0])
        if len(result) == len(merged_testing_label):
            print '# of testing data', len(result)
            print '# of true positives', len([i for i, j in zip(result, merged_testing_label) if i == j ==0])
            print '# of false positives', result.count(0) - len([i for i, j in zip(result, merged_testing_label) if i == j == 0])
            print '# of true negatives', len([i for i, j in zip(result, merged_testing_label) if i == j ==1])
            print '# of false negatives', result.count(1) - len([i for i, j in zip(result, merged_testing_label) if i == j ==1])
            print float(len([i for i, j in zip(result, merged_testing_label) if i == j]))/len(result)
            num_0 = len([i for i, j in zip(result, merged_testing_label) if i == j == 0]) # all detected 0
            num_1 = len([i for i, j in zip(result, merged_testing_label) if i == j == 1]) # all detected 1

            label_0 = merged_testing_label.count(0) # all label 0
            label_1 = merged_testing_label.count(1) # all label 1

            result_0 = result.count(0) # all 0 in result
            result_1 = result.count(1) # all 1 in result
            print num_0,num_1,result_0,result_1
            accuracy = float(num_0+num_1)/len(result)

            Precision =  float(num_1)/result_1
            recall = float(num_1)/label_1
#             Precision =  float(num_0)/result_0
#             recall = float(num_0)/label_0
            Fscore = (2*Precision*recall)/(recall+Precision)


            SUM_accuracy += accuracy
            SUM_Precision += Precision
            SUM_recall += recall
            SUM_Fscore += Fscore


        else:
            print 'exception'
        
print 'Accuracy', SUM_accuracy/20
print 'Precision', SUM_Precision/20
print 'Recall', SUM_recall/20
print 'F-score', SUM_Fscore/20
    

# of testing data 670
# of true positives 31
# of false positives 9
# of true negatives 411
# of false negatives 219
0.659701492537
31 411 40 630
# of testing data 669
# of true positives 23
# of false positives 17
# of true negatives 403
# of false negatives 226
0.636771300448
23 403 40 629
# of testing data 670
# of true positives 32
# of false positives 8
# of true negatives 412
# of false negatives 218
0.662686567164
32 412 40 630
# of testing data 669
# of true positives 23
# of false positives 17
# of true negatives 403
# of false negatives 226
0.636771300448
23 403 40 629
# of testing data 670
# of true positives 30
# of false positives 9
# of true negatives 411
# of false negatives 220
0.658208955224
30 411 39 631
# of testing data 670
# of true positives 35
# of false positives 17
# of true negatives 403
# of false negatives 215
0.653731343284
35 403 52 618
# of testing data 670
# of true positives 35
# of false positives 9
# of true negatives 411
# of false negatives 215
0.66